In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from mgen import rotation_from_angle
from IPython.display import clear_output
from ipywidgets import Button, FloatSlider, HBox, VBox, IntSlider, Output, interact
from affine_transform import transform

In [2]:
def overlay_RGB(I_list):
    Z, axarr = plt.subplots(1, len(I_list) - 1, figsize=(25,10))
    for i in range(len(I_list) - 1):
        overlay = np.zeros((I_list[i].shape) + (3,), dtype=float)
        overlay[:, :, 0] = I_list[i].astype(float)
        overlay[:, :, 1] = I_list[i + 1].astype(float)
        overlay[:, :, 2] = (
            I_list[i].astype(float) * 0.5 + I_list[i + 1].astype(float) * 0.5
        )
        if (len(I_list) - 1) == 1:
            axarr.imshow(overlay / np.max(overlay))
        else:
            axarr[i].imshow(overlay / np.max(overlay))
    plt.show()

In [5]:
#file paths
filepath = "/home/yilmaz/public/09 Staff/Yilmaz/for Penny/new/"
file1 = filepath+"2020_03_03__FL_mCherry__20x_041.tiff" #(0,10)#lower resolution image to avoid further pixelation
# file2 = filepath+"2020_03_03__FL_ELF97_20x_0012.tif" #(median,max)
# file2 = filepath+"2020_03_03__BF_and_TB_20x_0046.tiff" #(0,max,invert)
file2 = filepath+"2020_03_04_BF_Basic_and_TB_20x_00060.tiff" #(0,median,invert)

invert = 1#if the 2nd image should be inverted, set to 1, otherwise set to 0

#read in files
image1_rgb = plt.imread(file1)
image2_rgb = plt.imread(file2)

#make 3D images 2D (comment out for 2D images- if binary, also multiply by 1)
image1=0.2989*image1_rgb[:,:,0] + 0.5870*image1_rgb[:,:,1] + 0.1140*image1_rgb[:,:,2]
image2=0.2989*image2_rgb[:,:,0] + 0.5870*image2_rgb[:,:,1] + 0.1140*image2_rgb[:,:,2]


#make images the same size
size1 = image1.shape
size2 = image2.shape
# print(size1, size2)
new_size = [np.max([size1[0],size2[0]]),np.max([size1[1],size2[1]])]

im1=np.zeros(new_size)
im1[0:size1[0],0:size1[1]]=image1
im2=np.zeros(new_size)
im2[0:size2[0],0:size2[1]]=image2

#scaling of the image visualization
# print(np.min(im1),np.max(im1),np.median(im1))
# print(np.min(im2),np.max(im2),np.median(im2))
image1_min = np.median(im1)
image1_max = 10#np.max(im1)
image2_min = 0#np.median(im2)
image2_max = np.max(im2)

im1[im1>image1_max]=image1_max
im1[im1<image1_min]=image1_min
im2[im2>image2_max]=image2_max
im2[im2<image2_min]=image2_min

#scale images to 0-1
im1 = (im1 - im1.min()) / (im1.max() - im1.min())
if invert==1:
    im2 = (im2.max()-im2) / (im2.max() - im2.min())
else:
    im2 = (im2 - im2.min()) / (im2.max() - im2.min())



params = [0, 0, 0, 0, 0, 0, 1]


rot = FloatSlider(
    value=params[2], description="CCW Rot", min=-90, max=90, step=0.1, readout_format=".1f"
)
tx = IntSlider(
    value=params[3], description="Down X", min=-5000, max=5000, step=10
)
ty = IntSlider(
    value=params[4], description="Right Y", min=-3000, max=3000, step=10
)
scale = FloatSlider(
    value=params[6], description="Scale", min=0.1, max=5, step=0.005, readout_format=".3f"
)



sliderbox = HBox([rot, tx, ty, scale])
display(sliderbox)

out = Output()
button = Button(description="Transform")
vbox = VBox(children=(out, button))
display(vbox)

transformed = np.zeros(im2.shape)


def transform_image(im, r, t1, t2, sc):
    global transformed
    regparams = [0, 0, r, t1, t2, 0, sc]
    if np.max(regparams[0:5]) == 0 and np.min(regparams[0:5]) == 0 and regparams[6]==1:
        transformed = np.copy(im)
    else:
        transformed = transform(im,np.dot(np.diag((sc,)*2),rotation_from_angle(np.radians(r))),[t1,t2],order="linear",origin=[0,0],background_value=0)
    adjust_view()
    return transformed


def adjust_view():
    images = [im1, transformed]
    with out:
        clear_output(wait=True)
        overlay_RGB(images)
    vbox.children = [button, out]


def click(event=None):
    transform_image(
        im2, rot.value, tx.value, ty.value, scale.value
    )


button.on_click(click)

interact(adjust_view)

FileNotFoundError: [Errno 2] No such file or directory: '/home/yilmaz/public/09 Staff/Yilmaz/for Penny/new/2020_03_03__FL_mCherry__20x_041.tiff'

In [6]:
#output image
paramfile = os.path.splitext(file2)[0]+"_transformed.txt"
pfile = open(paramfile,"a") 
pfile.write(str(rot.value)+', '+str(round(tx.value))+', '+str(round(ty.value))+', '+str(scale.value)+'\n')
pfile.close() 

outputfile = os.path.splitext(file2)[0]+"_transformed.tiff"
image2_write = np.zeros(image1_rgb.shape)
if size1[0]>size2[0] & size1[1]>size2[1]:
    image2_write[0:image2_rgb.shape[0],0:image2_rgb.shape[1],:]=image2_rgb[:,:,:]
elif size1[0]>size2[0] & size1[1]<size2[1]:
    image2_write[:,0:image2_rgb.shape[1],:]=image2_rgb[:,0:image1_rgb.shape[1],:]
elif size1[0]<size2[0] & size1[1]>size2[1]:
    image2_write[0:image2_rgb.shape[0],:,:]=image2_rgb[0:image1_rgb.shape[0],:,:]
else:
    image2_write[:,:,:]=image2_rgb[0:image1_rgb.shape[0],0:image1_rgb.shape[1],:]

print('transforming red...')
trans_r=np.round(transform(image2_write[:,:,0], np.dot(np.diag((scale.value,)*2),rotation_from_angle(np.radians(rot.value))), [tx.value,ty.value], order="cubic", origin=[0,0], background_value=0)).astype(np.uint8)
trans_r[trans_r<0]=0
trans_r[trans_r>255]=255

print('transforming green...')
trans_g=np.round(transform(image2_write[:,:,1], np.dot(np.diag((scale.value,)*2),rotation_from_angle(np.radians(rot.value))), [tx.value,ty.value], order="cubic", origin=[0,0], background_value=0)).astype(np.uint8)
trans_g[trans_g<0]=0
trans_g[trans_g>255]=255
print('transforming blue...')
trans_b=np.round(transform(image2_write[:,:,2], np.dot(np.diag((scale.value,)*2),rotation_from_angle(np.radians(rot.value))), [tx.value,ty.value], order="cubic", origin=[0,0], background_value=0)).astype(np.uint8)
trans_b[trans_b<0]=0
trans_b[trans_b>255]=255

print('saving transformed image...')
plt.imsave(outputfile,np.dstack((trans_r,trans_g,trans_b)))


transforming red...
transforming green...
transforming blue...
saving transformed image...
